## Introduction : Business Problem

I have a plan to build a coffee shop in New York City, but still have no idea where the best place should I build my coffee shop is
Location is one of the most important factors in building my business, and make sure that my new coffee shop will be reachable/ close to the market or potential customers

## Data Description

1. I will use the data that I've gained from the previous assignment (New York City) and use the Foursquare Data API to understand the neighbourhood, e.g : office areas, restaurants, coffee shop, school, universities etc
2. As I target my potential customers are the university students, I will explore in more detail the university venue in the Foursquare data API, in which area has the most number of universities, and explore how many coffee shops are available around the area (< 1 Km) from the universities
3. The potential location to build my coffee shops will be in the area that has many universities but still lack of coffee shops available near the university's neighbourhood
4. I will also use other information regarding other types of venues that are mostly common in the area and use the clustering algorithm to get a slight picture about the major interests of the society in that area

In [2]:
#download all the dependencies/ libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 93 kB 2.3 MB/s  eta 0:00:01
Libraries imported.
